In [2]:
import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd
import time
import pandas as pd
import plotly.express as px
import plotly.io as pio
import scipy.io.wavfile as wav
pio.renderers.default = "notebook"

In [3]:
from pysonance.signal import *
from pysonance.const import *
from pysonance.sound import *
from pysonance.control import *
from pysonance.filter import *
from pysonance.sampler import *
from pysonance.pactl import *

Mostrar ondas copiandolas para que no se altere el frame.

In [4]:

from copy import deepcopy
def showOsc(osc, time, chunks=False):
    _osc = deepcopy(osc)
    signal = np.zeros(0)
    _chunks = int(time*(SRATE+CHUNK)/CHUNK)
    if chunks:
        _chunks = time
    for i in range(_chunks):
        signal = np.concatenate((signal, _osc.next()))
    plt.plot(signal)

In [5]:
'''REPRODUCTOR'''
input = None

def callback(outdata, frames, time, status):
    global input
    # print('entro')
    if input is not None:
        bloque = input.next()
        # convertimos formato (CHUNK,) a (CHUNK,1) para que adecuarlo a sounddevice
        outdata[:] = bloque.reshape(-1, 1)
    else:
        # si no hay datos, reproducimos silencio
        outdata[:] = np.zeros((CHUNK, 1))


# stream de salida con callBack
stream = sd.OutputStream(samplerate=SRATE, channels=2, callback=callback, blocksize=CHUNK)
stream.start()

In [6]:
def play(osc:Signal, add=False):
    global input 
    if add is False or input is None:
        input = osc # sustituye lo que estaba sonando antes
    else:
        input += osc # añade a lo que estaba sonando antes
        
def stop():
    global input 
    input = None
    
def playFor(osc:Signal, secs=5, add=False):
    play(osc)
    time.sleep(secs)
    stop()   
    
def live(osc:Signal, secs=2):
    ''' Devuelve los chunks concatenados como si se hubieran sacado live'''
    wave = np.array([])
    for i in range(int(secs*SRATE/CHUNK)):
        chunk = osc.next(CHUNK)
        wave = np.concatenate((wave, chunk))
    return wave 
    
def show(osc:Signal, secs=.5):
    plt.plot(osc.next(SRATE*secs))
    
def show2(osc:Signal, secs=.5):
    df = pd.DataFrame({"amp": osc.next(SRATE*secs)})
    df["secs"] = df.index / SRATE
    wave = px.line(df, y="amp", title="Waveform")
    wave.show()
    
def show_live(osc:Signal, secs=.5):
    wave = live(osc, secs)
    plt.plot(wave)
    
def show2_live(osc:Signal, secs=.5):
    wave = live(osc, secs)
    df = pd.DataFrame({"amp": wave})
    df["secs"] = df.index / SRATE
    wave = px.line(df, y="amp", title="Waveform")
    wave.show()

    

In [7]:
def toWav(osc:Signal, filename="output.wav", secs=5):
    wave = osc.next(SRATE*secs)
    wave = (wave * 32767).astype(np.int16)  # Convertir a formato PCM 16 bits
    wav.write(filename, SRATE, wave)
    
def toWav_live(osc:Signal, filename="output.wav", secs=5):
    wave = np.array([])
    for i in range(int(secs*SRATE/CHUNK)):
        chunk = osc.next(CHUNK)
        wave = np.concatenate((wave, chunk))
    wave = (wave * 32767).astype(np.int16)  # Convertir a formato PCM 16 bits
    wav.write(filename, SRATE, wave)
        
    

Para cargar librerias y reproducir sonido

In [8]:
raise Exception("Para de ejecutar!!")

Exception: Para de ejecutar!!

In [ ]:
playFor(Sine(440), 1)

In [ ]:
playFor(Noise(), 1)

uso de los wavetables

In [ ]:
s2 = Sine(1, phase=1).next(SRATE)[-1]
print(s2)

In [ ]:
s2 = Sine(1, phase=1)
s = Sine(1).next(SRATE)
wt = WT(s, freq=1, phase=1)
# toWav(wt, 'sine_table.wav', secs=5)
show_live(s2, 1)
show_live(wt, 1)



In [ ]:
s = Sine(1).next(SRATE*4)
samp = Sampler(s, 1, loop = True)
samp2 = Sampler(s, 1, loop = False)
show_live(samp, 6)
show(samp2,6)

In [ ]:
playFor(wt, 3)
playFor(Sine(440), 3)

In [ ]:
for i, dev in enumerate(sd.query_devices()):
    if dev['max_input_channels'] > 0:
        print(i, dev['name'])

0 Jabra EVOLVE 20 SE MS: USB Audio (hw:0,0)
6 HD-Audio Generic: ALC887-VD Analog (hw:2,0)
8 HD-Audio Generic: ALC887-VD Alt Analog (hw:2,2)
9 sysdefault
13 spdif
14 lavrate
15 samplerate
16 speexrate
17 jack
18 pipewire
19 pulse
21 speex
22 upmix
23 vdownmix
25 default
26 Jabra EVOLVE 20 SE MS Mono
27 Starship/Matisse HD Audio Controller Analog Stereo
28 3-Band Splitter
29 spotify
30 Navi 10 HDMI Audio Digital Stereo (HDMI 5)
31 Jabra EVOLVE 20 SE MS Analog Stereo
32 prueba Audio/Source/Virtual sink-87
33 prueba Audio/Source/Virtual sink-90
34 prueba Audio/Source/Virtual sink
35 prueba Audio/Source/Virtual sink-99
36 prueba Audio/Source/Virtual sink-105
37 prueba Audio/Source/Virtual sink-117
38 p2222 Audio/Source/Virtual sink
39 prueba2 Audio/Source/Virtual sink
40 prueba2 Audio/Source/Virtual sink-131
41 prueba Audio/Source/Virtual sink-136
42 30 Audio/Source/Virtual sink
43 prueba Audio/Source/Virtual sink-144
44 prueba Audio/Source/Virtual sink-146
45 prueba Audio/Source/Virtual si

In [9]:
m = Line_In('prueba Audio/Source/Virtual sink') * 3
play(Sine(440, amp=m))

In [21]:
show2_live(m, 1)

input overflow, output underflow
